In [ ]:
target = "/mnt/bn/nlhei-nas/liubangya/proj/vlm/workspace/image_hint_9x16_2/results/ans_base.json"
import numpy as np
import json

def dictify(s):
    s = s.split(", ")
    ret_dict = {}
    for ss in s:
        triplet = ss.split(' ')
        if len(triplet) != 3:
            return None
        try:
            triplet = [int(triplet[0]), int(triplet[1]), float(triplet[2])]
        except:
            return None
        ret_dict[triplet[0]] = triplet[1:]
    return ret_dict


MAX_ERROR_PER_FRAME = 5

def eval(ans_path):
    with open(ans_path, "r") as f:
        lines = f.readlines()
    total_error = 0
    per_frame_stat = {}
    for l in lines:
        data = json.loads(l)
        gt = data["gt_ans"]
        pred = data["ans"]
        gt = dictify(gt)
        pred = dictify(pred)

        error_per_ans = 0
        if pred is None:
            error_per_ans = 1
            total_error += error_per_ans
            continue

        for k in gt.keys():
            if k not in pred:
                error_per_ans += 1
                per_frame_stat.setdefault(k, []).append(1)
            else:
                dist = (pred[k][0] - gt[k][0])**2 + (pred[k][1] - gt[k][1])**2
                dist = min(dist**0.5, MAX_ERROR_PER_FRAME)
                error_per_ans += dist / MAX_ERROR_PER_FRAME
                per_frame_stat.setdefault(k, []).append(dist / MAX_ERROR_PER_FRAME)
        
        error_per_ans /= len(gt)
        total_error += error_per_ans
    
    per_frame_stat = {k: float(np.mean(v)) * MAX_ERROR_PER_FRAME for k, v in per_frame_stat.items()}
    return total_error, per_frame_stat




In [11]:
import glob
import os

dirs = glob.glob("/mnt/bn/nlhei-nas/liubangya/proj/vlm/workspace/image_*")
for d in dirs:
    ans = glob.glob(
        os.path.join(d, "results/*.json")
    )
    for a in ans:
        exp_name = os.path.basename(d)
        setup_name = os.path.basename(a).replace(".json", "")
        score, score_per_frame = eval(a)
        print(f"""
        {exp_name} {setup_name} >>>
        {score}
        {score_per_frame}
        """)


        image_hint_9x16_2 ans_base >>>
        395.5076527483811
        {0: 0.06266135174586972, 1: 0.681151234429223}
        

        image_hint_9x16_2 ans_lora >>>
        197.33687559773904
        {0: 0.0, 1: 0.493342188994348}
        

        image_hint_9x16_2_indexed ans_base >>>
        528.4729980922411
        {0: 0.18953545745626016, 1: 0.7539392432917581}
        

        image_hint_9x16_2_indexed ans_lora >>>
        189.67329452686644
        {0: 0.0, 1: 0.4741832363171662}
        

        image_hint_9x16_4 ans_base >>>
        766.0582250348307
        {0: 0.8520253128566592, 1: 0.9273300000587125, 2: 0.9616794058296336, 3: 0.9908524291197238}
        

        image_hint_9x16_4 ans_lora >>>
        333.52739654106625
        {0: 0.0, 1: 0.42782326463223774, 2: 0.6123501045502308, 3: 0.724949021152774}
        

        image_hint_9x16_4_indexed ans_base >>>
        764.1912318873243
        {0: 0.8437867224780785, 1: 0.9227208816350373, 2: 0.9590604036821289, 3: